In [31]:
%load_ext nb_black
# !pip install nb_black

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

In [32]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import metrics as m
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE  # doctest: +NORMALIZE_WHITESPACE
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
import xgboost as xgb
from sklearn.pipeline import make_pipeline
from mlxtend.feature_selection import ColumnSelector
from sklearn import model_selection
from mlxtend.classifier import StackingClassifier
from thundersvm import SVC as svmgpu

import warnings

warnings.filterwarnings("ignore")
randomseed = 7
np.random.seed(randomseed)

<IPython.core.display.Javascript object>

# 1. Read the dataset

In [33]:
x_original = pd.read_csv("XLable_onlyDiabeticRemoved.txt")

conditions = [
    (x_original.L100800 < 100),
    (x_original.L100800 >= 100) & (x_original.L100800 < 126),
    (x_original.L100800 >= 126),
]
choices = [0, 1, 2]
x_original["CLASS"] = np.select(conditions, choices, default=0)
x_original = x_original[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "S000300",
        "AGE",
        "SEX",
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
        "FIELD_1",
    ]
]

<IPython.core.display.Javascript object>

In [34]:
x_original2 = pd.read_csv("XLable_onlyDiabeticRemoved2.txt")

conditions = [
    (x_original2.L100800 < 100),
    (x_original2.L100800 >= 100) & (x_original2.L100800 < 126),
    (x_original2.L100800 >= 126),
]
choices = [0, 1, 2]
x_original2["CLASS"] = np.select(conditions, choices, default=0)
x_original2 = x_original2[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "S000300",
        "AGE",
        "SEX",
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
        "FIELD_1",
    ]
]

print(x_original2.shape)

(20414, 13)


<IPython.core.display.Javascript object>

In [35]:
x_original3 = pd.read_csv("XLable_onlyDiabeticRemoved3.txt")

conditions = [
    (x_original3.L100800 < 100),
    (x_original3.L100800 >= 100) & (x_original3.L100800 < 126),
    (x_original3.L100800 >= 126),
]

choices = [0, 1, 2]
x_original3["CLASS"] = np.select(conditions, choices, default=0)

x_original3 = x_original3[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "S000300",
        "AGE",
        "SEX",
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
        "FIELD_1",
    ]
]

print(x_original3.shape)

(20414, 13)


<IPython.core.display.Javascript object>

In [36]:
x_original4 = pd.read_csv("XLable_onlyDiabeticRemoved4.txt")

conditions = [
    (x_original4.L100800 < 100),
    (x_original4.L100800 >= 100) & (x_original4.L100800 < 126),
    (x_original4.L100800 >= 126),
]

choices = [0, 1, 2]
x_original4["CLASS"] = np.select(conditions, choices, default=0)

x_original4 = x_original4[
    [
        "Unnamed: 0",
        "L100800",
        "L104600",
        "S000300",
        "AGE",
        "SEX",
        "FIELD_16",
        "FIELD_23",
        "FIELD_15",
        "FIELD_22",
        "FIELD_17",
        "FIELD_24",
        "FIELD_1",
    ]
]

x_original4 = x_original4.rename(
    columns={
        "Unnamed: 0": "Unnamed: 0",
        "L100800": "L100800_z",
        "L104600": "L104600_z",
        "S000300": "S000300_z",
        "AGE": "AGE_z",
        "SEX": "SEX_z",
        "FIELD_16": "FIELD_16_z",
        "FIELD_23": "FIELD_23_z",
        "FIELD_15": "FIELD_15_z",
        "FIELD_22": "FIELD_22_z",
        "FIELD_17": "FIELD_17_z",
        "FIELD_24": "FIELD_24_z",
        "FIELD_1": "FIELD_1",
    }
)

print(x_original4.shape)

(20414, 13)


<IPython.core.display.Javascript object>

In [37]:
y_original = pd.read_csv("TargetLable_onlyDiabeticRemoved.txt")

conditions = [
    (y_original.L100800 < 100),
    (y_original.L100800 >= 100) & (y_original.L100800 < 126),
    (y_original.L100800 >= 126),
]

choices = [0, 1, 2]
y_original["CLASS"] = np.select(conditions, choices, default=0)

y_original = y_original[["Unnamed: 0", "FIELD_1", "CLASS"]]

print(y_original.shape)

(20414, 3)


<IPython.core.display.Javascript object>

In [38]:
data = pd.merge(
    x_original,
    x_original2,
    how="inner",
    left_on=["Unnamed: 0", "FIELD_1"],
    right_on=["Unnamed: 0", "FIELD_1"],
)
data.head()

,Unnamed: 0,L100800_x,L104600_x,S000300_x,AGE_x,SEX_x,FIELD_16_x,FIELD_23_x,FIELD_15_x,FIELD_22_x,...,L104600_y,S000300_y,AGE_y,SEX_y,FIELD_16_y,FIELD_23_y,FIELD_15_y,FIELD_22_y,FIELD_17_y,FIELD_24_y
0,0,78.0,NaN,23.5,67.0,1.0,0.0,0.0,0.0,0.0,...,NaN,23.2,68.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1,91.0,NaN,25.4,57.0,1.0,0.0,0.0,0.0,0.0,...,NaN,24.2,58.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,121.0,NaN,23.5,59.0,1.0,0.0,0.0,1.0,1.0,...,NaN,23.4,60.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0
3,3,96.0,NaN,18.3,16.0,1.0,0.0,0.0,0.0,0.0,...,NaN,18.7,17.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,84.0,5.0,19.9,19.0,1.0,0.0,0.0,0.0,0.0,...,5.3,20.3,20.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

In [39]:
data = pd.merge(
    data,
    x_original3,
    how="inner",
    left_on=["Unnamed: 0", "FIELD_1"],
    right_on=["Unnamed: 0", "FIELD_1"],
)
data.head()

,Unnamed: 0,L100800_x,L104600_x,S000300_x,AGE_x,SEX_x,FIELD_16_x,FIELD_23_x,FIELD_15_x,FIELD_22_x,...,L104600,S000300,AGE,SEX,FIELD_16,FIELD_23,FIELD_15,FIELD_22,FIELD_17,FIELD_24
0,0,78.0,NaN,23.5,67.0,1.0,0.0,0.0,0.0,0.0,...,5.97,23.5,69.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1,91.0,NaN,25.4,57.0,1.0,0.0,0.0,0.0,0.0,...,5.60,24.6,59.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,121.0,NaN,23.5,59.0,1.0,0.0,0.0,1.0,1.0,...,6.21,24.2,61.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,3,96.0,NaN,18.3,16.0,1.0,0.0,0.0,0.0,0.0,...,NaN,18.9,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,84.0,5.0,19.9,19.0,1.0,0.0,0.0,0.0,0.0,...,5.06,20.4,21.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

In [40]:
data = pd.merge(
    data,
    x_original4,
    how="inner",
    left_on=["Unnamed: 0", "FIELD_1"],
    right_on=["Unnamed: 0", "FIELD_1"],
)
data.head()

,Unnamed: 0,L100800_x,L104600_x,S000300_x,AGE_x,SEX_x,FIELD_16_x,FIELD_23_x,FIELD_15_x,FIELD_22_x,...,L104600_z,S000300_z,AGE_z,SEX_z,FIELD_16_z,FIELD_23_z,FIELD_15_z,FIELD_22_z,FIELD_17_z,FIELD_24_z
0,0,78.0,NaN,23.5,67.0,1.0,0.0,0.0,0.0,0.0,...,6.05,23.1,70.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
1,1,91.0,NaN,25.4,57.0,1.0,0.0,0.0,0.0,0.0,...,5.58,24.1,60.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,2,121.0,NaN,23.5,59.0,1.0,0.0,0.0,1.0,1.0,...,6.77,24.7,62.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
3,3,96.0,NaN,18.3,16.0,1.0,0.0,0.0,0.0,0.0,...,NaN,18.6,19.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,84.0,5.0,19.9,19.0,1.0,0.0,0.0,0.0,0.0,...,5.02,20.0,22.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

In [41]:
data = pd.merge(
    data,
    y_original,
    how="inner",
    left_on=["Unnamed: 0", "FIELD_1"],
    right_on=["Unnamed: 0", "FIELD_1"],
)

<IPython.core.display.Javascript object>

In [42]:
# filter the data set
data = data[data.FIELD_16_x != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23_x != 1]  # exclude people who are on medication for diabetes

data = data[data.FIELD_16_y != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23_y != 1]  # exclude people who are on medication for diabetes

data = data[data.FIELD_16 != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23 != 1]  # exclude people who are on medication for diabetes

data = data[data.FIELD_16_z != 1]  # exclude people who are diagnosed for (diabetes)
data = data[data.FIELD_23_z != 1]  # exclude people who are on medication for diabetes

# data = data[
#     data.FIELD_15_x != 1
# ]  # exclude people who are diagnosed for (high blood pressure)
# data = data[
#     data.FIELD_22_x != 1
# ]  # exclude people who are on medication for high blood pressure

# data = data[
#     data.FIELD_15_y != 1
# ]  # exclude people who are diagnosed for (high blood pressure)
# data = data[
#     data.FIELD_22_y != 1
# ]  # exclude people who are on medication for high blood pressure

# data = data[data.FIELD_17_x != 1]  # exclude people who are diagnosed for hyperlipidemia
# data = data[
#     data.FIELD_24_x != 1
# ]  # exclude people who are on medication for hyperlipidemia

# data = data[data.FIELD_17_y != 1]  # exclude people who are diagnosed for hyperlipidemia
# data = data[
#     data.FIELD_24_y != 1
# ]  # exclude people who are on medication for hyperlipidemia

# print(data.shape)

<IPython.core.display.Javascript object>

In [43]:
data.head()

,Unnamed: 0,L100800_x,L104600_x,S000300_x,AGE_x,SEX_x,FIELD_16_x,FIELD_23_x,FIELD_15_x,FIELD_22_x,...,S000300_z,AGE_z,SEX_z,FIELD_16_z,FIELD_23_z,FIELD_15_z,FIELD_22_z,FIELD_17_z,FIELD_24_z,CLASS
0,0,78.0,NaN,23.5,67.0,1.0,0.0,0.0,0.0,0.0,...,23.1,70.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1
1,1,91.0,NaN,25.4,57.0,1.0,0.0,0.0,0.0,0.0,...,24.1,60.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
2,2,121.0,NaN,23.5,59.0,1.0,0.0,0.0,1.0,1.0,...,24.7,62.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1
3,3,96.0,NaN,18.3,16.0,1.0,0.0,0.0,0.0,0.0,...,18.6,19.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,4,84.0,5.0,19.9,19.0,1.0,0.0,0.0,0.0,0.0,...,20.0,22.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0


<IPython.core.display.Javascript object>

In [44]:
data = data.dropna()
print(data.shape)
data = data[
    [
        "L100800_x",
        "L104600_x",
        "S000300_x",
        "AGE_x",
        "SEX_x",
        "L100800_y",
        "L104600_y",
        "S000300_y",
        "AGE_y",
        "SEX_y",
        "L100800",
        "L104600",
        "S000300",
        "AGE",
        "SEX",
        "L100800_z",
        "L104600_z",
        "S000300_z",
        "AGE_z",
        "SEX_z",
        "CLASS",
    ]
]
data.head()

(5730, 47)


,L100800_x,L104600_x,S000300_x,AGE_x,SEX_x,L100800_y,L104600_y,S000300_y,AGE_y,SEX_y,...,L104600,S000300,AGE,SEX,L100800_z,L104600_z,S000300_z,AGE_z,SEX_z,CLASS
4,84.0,5.0,19.9,19.0,1.0,86.0,5.3,20.3,20.0,1.0,...,5.06,20.4,21.0,1.0,80.0,5.02,20.0,22.0,1.0,0
5,81.0,4.9,18.4,19.0,1.0,70.0,4.9,18.6,20.0,1.0,...,5.11,18.8,21.0,1.0,77.0,5.23,18.9,22.0,1.0,0
6,90.0,5.2,18.7,20.0,1.0,89.0,5.3,20.0,21.0,1.0,...,5.13,18.9,22.0,1.0,88.0,4.82,17.8,23.0,1.0,1
9,92.0,5.4,17.4,20.0,1.0,95.0,5.4,18.0,21.0,1.0,...,5.28,16.9,22.0,1.0,80.0,5.61,16.3,23.0,1.0,0
14,91.0,4.8,21.6,22.0,1.0,87.0,5.1,22.3,23.0,1.0,...,4.82,21.9,24.0,1.0,91.0,4.93,22.0,25.0,1.0,1


<IPython.core.display.Javascript object>

In [58]:
data["L100800diff"] = np.power(data.L100800_y - data.L100800_x, 1)
data["L104600diff"] = np.power(data.L104600_y - data.L104600_x, 1)
data["S000300diff"] = np.power(data.S000300_y - data.S000300_x, 1)

data["L100800diff1"] = np.power(data.L100800 - data.L100800_y, 1)
data["L104600diff1"] = np.power(data.L104600 - data.L104600_y, 1)
data["S000300diff1"] = np.power(data.S000300 - data.S000300_y, 1)

data["L100800diff2"] = np.power(data.L100800_z - data.L100800, 1)
data["L104600diff2"] = np.power(data.L104600_z - data.L104600, 1)
data["S000300diff2"] = np.power(data.S000300_z - data.S000300, 1)

# data["L100800diff2"] = np.power(data.L100800_z - data.L100800_y, 1)
# data["L104600diff2"] = np.power(data.L104600_z - data.L104600_y, 1)
# data["S000300diff2"] = np.power(data.S000300_z - data.S000300_y, 1)


tempclass = data.CLASS
data = data.drop(columns="CLASS")
data["CLASS"] = tempclass

<IPython.core.display.Javascript object>

In [59]:
data.head()

,L100800_x,L104600_x,S000300_x,AGE_x,SEX_x,L100800_y,L104600_y,S000300_y,AGE_y,SEX_y,...,L100800diff,L104600diff,S000300diff,L100800diff1,L104600diff1,S000300diff1,L100800diff2,L104600diff2,S000300diff2,CLASS
4,84.0,5.0,19.9,19.0,1.0,86.0,5.3,20.3,20.0,1.0,...,2.0,0.3,0.4,1.0,-0.24,0.1,-7.0,-0.04,-0.4,0
5,81.0,4.9,18.4,19.0,1.0,70.0,4.9,18.6,20.0,1.0,...,-11.0,0.0,0.2,5.0,0.21,0.2,2.0,0.12,0.1,0
6,90.0,5.2,18.7,20.0,1.0,89.0,5.3,20.0,21.0,1.0,...,-1.0,0.1,1.3,-4.0,-0.17,-1.1,3.0,-0.31,-1.1,1
9,92.0,5.4,17.4,20.0,1.0,95.0,5.4,18.0,21.0,1.0,...,3.0,0.0,0.6,-17.0,-0.12,-1.1,2.0,0.33,-0.6,0
14,91.0,4.8,21.6,22.0,1.0,87.0,5.1,22.3,23.0,1.0,...,-4.0,0.3,0.7,18.0,-0.28,-0.4,-14.0,0.11,0.1,1


<IPython.core.display.Javascript object>

# 2. Downsample the majority class and upsample the minority

In [60]:
diabetic = data[data.CLASS == 2]
prediabetic = data[data.CLASS == 1]
normal = data[data.CLASS == 0]

print(diabetic.shape[0], prediabetic.shape[0], normal.shape[0])

169 2349 3212


<IPython.core.display.Javascript object>

In [61]:
diabetic_test = diabetic.sample(34, random_state=randomseed)
prediabetic_test = prediabetic.sample(34, random_state=randomseed)
normal_test = normal.sample(34, random_state=randomseed)
test = pd.concat([diabetic_test, prediabetic_test, normal_test])

diabetic_train = diabetic.drop(diabetic_test.index)
prediabetic_train = prediabetic.drop(prediabetic_test.index)
# .sample(
#     10 * diabetic_train.shape[0], random_state=randomseed
# )
normal_train = normal.drop(normal_test.index).sample(
    prediabetic_train.shape[0],
    random_state=randomseed
    #     10 * diabetic_train.shape[0], random_state=randomseed
)
train = pd.concat([diabetic_train, diabetic_train, prediabetic_train, normal_train])

<IPython.core.display.Javascript object>

In [62]:
xtrain = train.iloc[:, :-1]
ytrain = train.iloc[:, -1]
xtest = test.iloc[:, :-1]
ytest = test.iloc[:, -1]

<IPython.core.display.Javascript object>

In [63]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
xtrain = scaler.fit_transform(xtrain)
xtest = scaler.transform(xtest)

<IPython.core.display.Javascript object>

In [64]:
from imblearn.over_sampling import SMOTE, SMOTENC  # doctest: +NORMALIZE_WHITESPACE

randomseed = 42

sm = SMOTENC(
    random_state=randomseed, categorical_features=[4], sampling_strategy="minority"
)
X_res, y_res = sm.fit_resample(xtrain, ytrain)

print("Resampled dataset shape %s" % Counter(y_res))
print(
    y_res[y_res == 0].shape[0], y_res[y_res == 1].shape[0], y_res[y_res == 2].shape[0]
)
print(X_res.shape, y_res.shape)

xtrain = X_res
ytrain = y_res

Resampled dataset shape Counter({2: 2315, 1: 2315, 0: 2315})
2315 2315 2315
(6945, 29) (6945,)


<IPython.core.display.Javascript object>

# 3. Generate the classifier models based on the selected 12 features

# 3.1.  Features : 12

# a. Random forest

In [22]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

parameters = {
    "max_features": ("auto", "sqrt", "log2"),
    "n_estimators": [10, 100, 200, 700],
    "max_depth": [2, 8, 10],
    "min_samples_split": [2, 8, 12],
    "min_samples_leaf": [2, 8, 12],
    "criterion": ["gini", "entropy"],
}

rf_clf = RandomForestClassifier(n_jobs=-1, verbose=1)
rf_clf = RandomizedSearchCV(rf_clf, parameters)
rf_clf.fit(xtrain, ytrain)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.3s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.6s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done   2 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=10)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=10)]: Using backend ThreadingBackend with 10 concurrent workers.
[Paral

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      |

RandomizedSearchCV(cv='warn', error_score='raise-deprecating',
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                                                    n_estimators='warn',
                                                    n_jobs

<IPython.core.display.Javascript object>

In [23]:
rf_clf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=10, max_features='log2', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=1,
                       warm_start=False)

<IPython.core.display.Javascript object>

In [65]:
rf_5 = RandomForestClassifier(
    random_state=randomseed,
    n_estimators=100,
    max_depth=12,
    min_samples_split=2,
    min_samples_leaf=10,
    max_features="auto",
)

# # rf_5=rf_clf.best_estimator_
# rf_5 = RandomForestClassifier(
#     bootstrap=True,
#     class_weight=None,
#     criterion="entropy",
#     max_depth=10,
#     max_features="log2",
#     max_leaf_nodes=None,
#     min_impurity_decrease=0.0,
#     min_impurity_split=None,
#     min_samples_leaf=2,
#     min_samples_split=2,
#     min_weight_fraction_leaf=0.0,
#     n_estimators=200,
#     n_jobs=-1,
#     oob_score=False,
#     random_state=randomseed,
#     verbose=1,
#     warm_start=False,
# )

<IPython.core.display.Javascript object>

In [66]:
rf_5.fit(xtrain, ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=12, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=10, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

<IPython.core.display.Javascript object>

In [67]:
ypred = rf_5.predict(xtest)
print("Accuracy = ", m.accuracy_score(ytest, ypred))

Accuracy =  0.6862745098039216


<IPython.core.display.Javascript object>

In [68]:
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

,Normal,Prediabetes,diabetes
Normal,28,6,0
Prediabetes,13,19,2
diabetes,0,11,23


<IPython.core.display.Javascript object>

In [69]:
print(m.classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.68      0.82      0.75        34
           1       0.53      0.56      0.54        34
           2       0.92      0.68      0.78        34

    accuracy                           0.69       102
   macro avg       0.71      0.69      0.69       102
weighted avg       0.71      0.69      0.69       102



<IPython.core.display.Javascript object>

In [70]:
from sklearn import model_selection

scores = model_selection.cross_val_score(
    rf_5, xtrain, ytrain, cv=10, scoring="accuracy"
)
print(
    "Accuracy: %0.2f (+/- %0.2f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "RandomForestClassifier", scores)
)

Accuracy: 0.79 (+/- 0.05) [RandomForestClassifier] 
 [[0.66666667 0.76005747 0.79885057 0.81609195 0.80028736 0.81673882
 0.82828283 0.79220779 0.76623377 0.81673882]]


<IPython.core.display.Javascript object>

# b. XGBOOST 

In [30]:
parameters = {
    "max_depth": [2, 8, 10],
    "n_estimators": [10, 100, 700],
    "learning_rate": [0.05, 0.15, 0.25],
    "min_child_weight": [1, 3, 5],
    "gamma": [0.0, 0.2, 0.4],
    "colsample_bytree": [0.3, 0.4, 0.5],
}
xgb_clf = xgb.XGBClassifier()
xgb_clf = RandomizedSearchCV(xgb_clf, parameters, verbose=2)
xgb_clf.fit(xtrain, ytrain)
xgb_clf.best_estimator_

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.05, gamma=0.2, colsample_bytree=0.5 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.05, gamma=0.2, colsample_bytree=0.5, total=   1.0s
[CV] n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.05, gamma=0.2, colsample_bytree=0.5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.9s remaining:    0.0s


[CV]  n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.05, gamma=0.2, colsample_bytree=0.5, total=   0.8s
[CV] n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.05, gamma=0.2, colsample_bytree=0.5 
[CV]  n_estimators=100, min_child_weight=5, max_depth=2, learning_rate=0.05, gamma=0.2, colsample_bytree=0.5, total=   0.8s
[CV] n_estimators=10, min_child_weight=1, max_depth=2, learning_rate=0.05, gamma=0.2, colsample_bytree=0.4 
[CV]  n_estimators=10, min_child_weight=1, max_depth=2, learning_rate=0.05, gamma=0.2, colsample_bytree=0.4, total=   0.1s
[CV] n_estimators=10, min_child_weight=1, max_depth=2, learning_rate=0.05, gamma=0.2, colsample_bytree=0.4 
[CV]  n_estimators=10, min_child_weight=1, max_depth=2, learning_rate=0.05, gamma=0.2, colsample_bytree=0.4, total=   0.1s
[CV] n_estimators=10, min_child_weight=1, max_depth=2, learning_rate=0.05, gamma=0.2, colsample_bytree=0.4 
[CV]  n_estimators=10, min_child_weight=1, max_depth=2, learning_rate=0.0

KeyboardInterrupt: 

<IPython.core.display.Javascript object>

In [71]:
import xgboost as xgb
from sklearn import metrics as m

xgb_model_5 = xgb.XGBClassifier(objective="binary:logistic", random_state=randomseed)
# objective="multi:softmax"
# objective="binary:logistic"
# xgb_model=xgb_clf.best_estimator_

# xgb_model_5=xgb_clf.best_estimator_
# xgb_model_5=xgb.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.3, gamma=0.0,
#               learning_rate=0.05, max_delta_step=0, max_depth=10,
#               min_child_weight=3, missing=None, n_estimators=700, n_jobs=1,
#               nthread=None, objective='multi:softprob', random_state=randomseed,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=None, subsample=1, verbosity=1)


<IPython.core.display.Javascript object>

In [72]:
xgb_model_5.fit(xtrain, ytrain)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

<IPython.core.display.Javascript object>

In [73]:
ypred = xgb_model_5.predict(xtest)
print("Accuracy = ", m.accuracy_score(ytest, ypred))

Accuracy =  0.6666666666666666


<IPython.core.display.Javascript object>

In [74]:
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

,Normal,Prediabetes,diabetes
Normal,27,7,0
Prediabetes,14,18,2
diabetes,0,11,23


<IPython.core.display.Javascript object>

In [75]:
print(m.classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.66      0.79      0.72        34
           1       0.50      0.53      0.51        34
           2       0.92      0.68      0.78        34

    accuracy                           0.67       102
   macro avg       0.69      0.67      0.67       102
weighted avg       0.69      0.67      0.67       102



<IPython.core.display.Javascript object>

In [76]:
scores = model_selection.cross_val_score(
    xgb_model_5,xtrain, ytrain, cv=10, scoring="accuracy"
)
print(
    "Accuracy: %0.2f (+/- %0.2f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "xgb_model", scores)
)


Accuracy: 0.75 (+/- 0.05) [xgb_model] 
 [[0.64367816 0.74425287 0.76724138 0.77155172 0.76867816 0.76767677
 0.78066378 0.73160173 0.69264069 0.81673882]]


<IPython.core.display.Javascript object>

# c. SVM

In [ ]:
from thundersvm import SVC as svmgpu

parameters = {
    "C": [0.1, 1, 10, 100, 1000],
    "gamma": [1, 0.1, 0.01, 0.001, 0.0001],
    "kernel": ["linear", "rbf"],
}

SVC_clf = svmgpu()
SVC_clf2 = RandomizedSearchCV(SVC_clf, parameters, verbose=2)
SVC_clf2.fit(xtrain, ytrain)


# sorted(SVC_clf2.cv_results_.keys())

In [ ]:
SVC_clf2.best_estimator_

In [77]:
# scv_5=SVC_clf2.best_estimator_
scv_5 = svmgpu(
    C=1,
    cache_size=None,
    class_weight={},
    coef0=0.0,
    decision_function_shape="ovo",
    degree=3,
    gamma=0.1,
    gpu_id=0,
    kernel="linear",
    max_iter=-1,
    max_mem_size=-1,
    n_jobs=-1,
    probability=False,
    random_state=randomseed,
    shrinking=False,
    tol=0.001,
    verbose=False,
)

<IPython.core.display.Javascript object>

In [78]:
# scv_5 = svmgpu(
#     C=70,
#     cache_size=200,
#     class_weight={},
#     coef0=0.0,
#     decision_function_shape="ovr",
#     degree=3,
#     gamma=0.001,
#     gpu_id=0,
#     kernel="linear",
#     max_iter=-1,
#     max_mem_size=-1,
#     n_jobs=-1,
#     probability=True,
#     random_state=42,
#     shrinking=True,
#     tol=0.001,
#     verbose=False,
# )

<IPython.core.display.Javascript object>

In [79]:
scv_5.fit(xtrain, ytrain)

SVC(C=1, cache_size=None, class_weight={}, coef0=0.0,
    decision_function_shape='ovo', degree=3, gamma=0.1, gpu_id=0,
    kernel='linear', max_iter=-1, max_mem_size=-1, n_jobs=-1, probability=False,
    random_state=42, shrinking=False, tol=0.001, verbose=False)

<IPython.core.display.Javascript object>

In [80]:
ypred = scv_5.predict(xtest)
print("Accuracy = ", m.accuracy_score(ytest, ypred))

Accuracy =  0.7058823529411765


<IPython.core.display.Javascript object>

In [81]:
confmatrx = pd.DataFrame(
    m.confusion_matrix(ytest, ypred),
    columns=["Normal", "Prediabetes", "diabetes"],
    index=["Normal", "Prediabetes", "diabetes"],
)
confmatrx.head()

,Normal,Prediabetes,diabetes
Normal,28,5,1
Prediabetes,11,19,4
diabetes,0,9,25


<IPython.core.display.Javascript object>

In [82]:
print(m.classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.72      0.82      0.77        34
           1       0.58      0.56      0.57        34
           2       0.83      0.74      0.78        34

    accuracy                           0.71       102
   macro avg       0.71      0.71      0.71       102
weighted avg       0.71      0.71      0.71       102



<IPython.core.display.Javascript object>

In [83]:
scores = model_selection.cross_val_score(
    scv_5, xtrain, ytrain, cv=10, scoring="accuracy"
)
print(
    "Accuracy: %0.2f (+/- %0.2f) [%s] \n [%s]"
    % (scores.mean(), scores.std(), "SVC_clf", scores)
)

Accuracy: 0.77 (+/- 0.03) [SVC_clf] 
 [[0.69827586 0.72701149 0.77729885 0.79310345 0.78448276 0.76911977
 0.81529582 0.75757576 0.77777778 0.77777778]]


<IPython.core.display.Javascript object>